### 이름 : 정경윤 
### YID : 2019162029 
### 과제명 : YBIGTA - 0707 크롤링
***

# <font color='royalblue'>1. 필요한 모듈 임포트</font>

In [1]:
%pip install selenium==4.8.2
%pip install webdriver_manager
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


# <font color='royalblue'>2. Selenium 이용하여 크롬 브라우저 열기</font>

In [3]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)
# 크롬 브라우저 버전과 크롬 매니저 버전 매치하는지 확인함 

In [4]:
codi_list = []

# 무신사 코디맵 페이지 이동 
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)

try: 
    driver.maximize_window()
except:
    pass

# <font color = 'royalblue'>3. 무신사 코디맵 정보 스크래핑</font>

In [5]:
# 코디맵 카테고리, 제목, 날짜, 조회수, 댓글수, 부연설명, 해시태그, url 칼럼 생성
columns_1 = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
# 코디맵 정보 담을 빈 리스트 생성
values_1 = []
# BeautifulSoup이 파싱할 객체 = 현 브라우저 (코디맵) / 파싱 방법 = lxml
soup = BeautifulSoup(driver.page_source, 'lxml')
# <li> tag에서 attribute가 class = 'style-list-item'인 모든 element들을 list로 반환 
data_rows = soup.find_all('li', attrs = {'class':'style-list-item'})

In [6]:
# 페이지를 가장 아래까지 스크롤하는 방법

interval = 1 # 1초에 한번씩 스크롤 내림

# 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script("return document.body.scrollHeight")
# document.body.scrollHeight : document.body element 전체 길이 
# execute_script() : 현재 웹페이지에서 javascript code 실행 


# 반복 수행

while True:
    # 스크롤 가장 아래로 내림 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    # 페이지 로딩 대기 
    time.sleep(interval)
    
    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height

print('스크롤 완료')

for i, row in enumerate(data_rows):                     # data_rows 리스트에 있는 값들과 인덱스를 차례대로 가져온다

    print(f'{i+1}번째 코디맵 정보 크롤링')
    blank = []
    
    # 코디맵 카테고리 가져오기 
    category = row.find('span', attrs = {'class':'style-list-information__text'})   # 카테고리 이름 가져오기 
    if category:                                                                    # 해당 조건에 맞는 정보가 하나라도 있을 경우:
        category = category.get_text().strip()                                      # html에서 txt만 가져온 뒤 공백 제거 
        blank.append(category)
    else:
        blank.append('Something is wrong')                                          # 해당 조건에 맞는 정보가 없는 경우:
        print(f'{i+1}번째 정보 가져올 때 문제 발생')
        continue
    
    # 코디명 가져오기 
    title = row.find('strong', attrs = {'class':'style-list-information__title'})   # 코디명 가져오기 
    if title:                                                                       # 해당 조건에 맞는 정보가 하나라도 있을 경우:
        title = title.get_text().strip()                                            # html에서 txt만 가져온 뒤 공백 제거 
        blank.append(title)
    else:
        blank.append('Something is wrong')                                          # 해당 조건에 맞는 정보가 없는 경우:
        print(f'{i+1}번째 정보 가져올 때 문제 발생')
        continue
    
    # 코디맵 날짜, 조회수, 댓글수 가져오기 
    post_info = row.find_all('span', attrs = {'class':'post-information__text'})    # 해당 조건에 맞는 정보를 리스트로 반환
    if post_info:
        date = post_info[0].get_text().strip()
        click = post_info[1].get_text().strip()[3:]
        comments = post_info[2].get_text().strip()[3:]
        blank.extend([date, click, comments])

    else:
        for i in range(3):
            blank.append('Something is wrong')
        print(f'{i+1}번째 정보 가져올 때 문제 발생')
        continue
    
    values_1.append(blank)
    print('---------------------------------------------------')        
    
    
df = pd.DataFrame(values_1, columns = columns_1)
codi_list.append(df)


스크롤 완료
1번째 코디맵 정보 크롤링
---------------------------------------------------
2번째 코디맵 정보 크롤링
---------------------------------------------------
3번째 코디맵 정보 크롤링
---------------------------------------------------
4번째 코디맵 정보 크롤링
---------------------------------------------------
5번째 코디맵 정보 크롤링
---------------------------------------------------
6번째 코디맵 정보 크롤링
---------------------------------------------------
7번째 코디맵 정보 크롤링
---------------------------------------------------
8번째 코디맵 정보 크롤링
---------------------------------------------------
9번째 코디맵 정보 크롤링
---------------------------------------------------
10번째 코디맵 정보 크롤링
---------------------------------------------------
11번째 코디맵 정보 크롤링
---------------------------------------------------
12번째 코디맵 정보 크롤링
---------------------------------------------------
13번째 코디맵 정보 크롤링
---------------------------------------------------
14번째 코디맵 정보 크롤링
---------------------------------------------------
15번째 코디맵 정보 크롤링
----------------------------------

In [7]:
df_ = pd.concat(codi_list).reset_index(drop=True) # index 초기화 해줘야함
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"16,092",49
1,캐주얼,블랙 매니아,21.02.23,"34,681",46
2,캐주얼,오늘의 코디,22.03.22,"12,205",38
3,댄디,추천 댄디 룩,21.09.15,"20,191",27
4,캐주얼,릴랙스 룩,21.02.23,"23,544",22
5,스트릿,힙하다 힙해,21.02.03,"35,295",21
6,캐주얼,심플한 코디,21.07.30,"39,591",18
7,댄디,데이트 코디,21.12.24,"10,316",18
8,댄디,훈훈 남친 룩,21.03.24,"19,350",16
9,캐주얼,지금 입기 좋아,21.08.31,"20,219",16


In [8]:
df_.shape

(60, 5)

# <font color = 'royalblue'>4. Dataframe을 csv 파일로 저장</font>

In [9]:
df_.to_csv('무신사_코디맵_크롤링.csv', encoding = 'utf-8-sig') 